In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import scienceplots

import time
import math
import os
import sys
import random
from functools import partial
from decimal import Decimal
import numpy as np
import scipy.io as sio
import pysindy as ps
from tqdm import trange

sys.path.insert(0, '../')
from utils import *
from solvel0 import solvel0, MIOSR
from best_subset import backward_refinement, brute_force_all_subsets
from UBIC import *
from bayesian_model_evidence import log_evidence

from skimage.restoration import estimate_sigma
import bm3d
from kneed import KneeLocator

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

from rdata import read_rds
from selective_inference import forward_stop_rule

from sklearn.preprocessing import StandardScaler
from sklearn import covariance
from abess import LinearRegression as AbessLinearRegression
from knockpy import KnockoffFilter, knockoff_stats, knockoffs
from knockpy.utilities import estimate_covariance
from scipy import stats
from statsmodels.stats.multitest import multipletests
from c2st.check import c2st # https://github.com/psteinb/c2st

alibi is not installed in the environment.


In [2]:
X_pre = np.load("../Cache/X_pre_kdv_noise50.npy")
y_pre = np.load("../Cache/y_pre_kdv_noise50.npy")
feature_names = np.load("../Cache/feature_names_kdv.npy")
fsInf = read_rds("../R/R_data/fsInf_screening_kdv_noise50.rds")

u_pre = y_pre.copy()

In [3]:
alpha = 0.01
adjusted_pvalues = fsInf.get("pv")
stop_step, false_discovery_rates = forward_stop_rule(adjusted_pvalues, alpha)
adjusted_pvalues = adjusted_pvalues[:stop_step+1]
rejections = np.sort((fsInf.get("vars")-1).astype(np.int32)[:stop_step+1])
feature_names[rejections]

array(['x0_1', 'x0_111', 'x0_1111', 'x0_11111', 'x0x0_1', 'x0^2x0_1',
       'x0^3x0_1', 'x0^4x0_1', 'x0^5x0_1', 'x0^6x0_1', 'x0x0_111',
       'x0^2x0_111', 'x0^3x0_111'], dtype='<U13')

In [4]:
X_pre_top = X_pre[:, rejections]
X_pre_top = X_pre_top/np.linalg.norm(X_pre_top, 2, axis=0)
y_pre = u_pre.copy()

In [5]:
best_subsets = solvel0(X_pre_top, y_pre, max_complexity=8, miosr=True, refine=True)
best_subsets = [tuple(best_subsets[-1][_] for _ in bs) 
                for bs in brute_force_all_subsets(X_pre_top[:, best_subsets[-1]], y_pre)[1]]

  0%|                                                                                                  | 0/8 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2026-04-04


100%|██████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 41.16it/s]


In [6]:
from selective_inference import sfs_si, stepwise_selective_inference, subset_fdr
import fpsample
from dppy.finite_dpps import FiniteDPP

from si4pipeline import (
                        construct_pipelines, 
                        extract_features, 
                        initialize_dataset, 
                        intersection, 
                        lasso, 
                        marginal_screening, 
                        stepwise_feature_selection, 
                        union, 
                        PipelineManager
                        )

In [7]:
np.random.seed(0); random.seed(0)
n_samples = 250*(1+1)
# max_fdr = 0.2; false_discovery_control_method = 'by'
max_fdr = 0.2; false_discovery_control_method = 'bh'
# max_fdr = 0.2; false_discovery_control_method =  None
for bs in best_subsets:
    fdrs = []
    for _ in range(len(y_pre)//n_samples):
        X_test = X_pre_top[:, bs]
        y_test = y_pre.ravel()
        
        np.random.seed(random.randint(0, 100))
        # sample_indices = sorted(set([np.random.randint(len(y_pre)) for _ in range(n_samples)]))
        sample_indices = fpsample.bucket_fps_kdline_sampling(X_test, n_samples=n_samples, h=3) # Farthest Point Sampling (FPS) is better!!!
        X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        # FPS + k-DPP
        DPP = FiniteDPP('likelihood', **{'L': X_test.dot(X_test.T)})
        DPP.flush_samples()
        for _ in range(n_samples//(len(bs))):
            DPP.sample_exact_k_dpp(size=len(bs))
        sample_indices = np.unique(np.ravel(DPP.list_of_samples))
        X_test = X_test[sample_indices]; y_test = y_test[sample_indices]
        
        manager = stepwise_selective_inference(support_size=X_test.shape[1])
        M, p_list = manager.inference(X_test, y_test, np.std(y_test))
        if false_discovery_control_method is not None:
            p_list = stats.false_discovery_control(p_list, method=false_discovery_control_method)
        # print(M, p_list, np.array(p_list) < 0.05)
        fdrs.append(subset_fdr(p_list))
        
    fdrs = np.array(fdrs)
    print(fdrs.mean(), stats.wilcoxon(fdrs-max_fdr, alternative='less').pvalue)

0.0 3.872108215522035e-06
1.833101300707942e-07 4.422457510605951e-05
0.011802492845296175 4.422457510605951e-05
0.07124358729785643 9.5367431640625e-07
0.9332865946395467 0.9999558380706846
1.3721347796135495 0.9999557754248939
1.7283710333353401 0.9999557754248939
2.041648913543911 0.9999557754248939
